In [5]:
import os
import time

import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v1"

EPOCHS = 10
BATCH_SIZES = [16, 32, 64, 128]
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']


def train_and_evaluate(batch_size, VERSION_INDEX):
    print(f"\n🔹 Entrenando con batch_size={batch_size} 🔹")

    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}{VERSION_INDEX}")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}"),
        ]
    )
    training_time = time.time() - start_time

    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}", exist_ok=True)
    history_df.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/history.csv", index=False)

    results = model.evaluate(test_images)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["batch_size"] = batch_size
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/results.csv", index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


all_results = []
count = 0
for batch_size in BATCH_SIZES:
    results_df = train_and_evaluate(batch_size, count)
    all_results.append(results_df)
    count += 1

final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/batch_comparison_results.csv", index=False)

print("\n✅ ¡Entrenamiento con diferentes batch sizes completado!")


🔹 Entrenando con batch_size=16 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


Epoch 1/10


2025-02-23 13:49:09.385998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1065/1066 [============================>.] - ETA: 0s - loss: 0.2327 - accuracy: 0.9007 - precision: 0.9244 - recall: 0.9496 - auc: 0.9510

2025-02-23 13:49:55.377006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

Epoch 1/10


2025-02-23 15:38:49.207547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - ETA: 0s - loss: 0.2944 - accuracy: 0.8753 - precision: 0.8911 - recall: 0.9561 - auc: 0.9169

2025-02-23 15:39:29.333181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 52s 376ms/step - loss: 0.2944 - accuracy: 0.8753 - precision: 0.8911 - recall: 0.9561 - auc: 0.9169 - val_loss: 0.2369 - val_accuracy: 0.9134 - val_precision: 0.9058 - val_recall: 0.9700 - val_auc: 0.9721
Epoch 2/10
134/134 [==============================] - 53s 398ms/step - loss: 0.2088 - accuracy: 0.9140 - precision: 0.9348 - recall: 0.9558 - auc: 0.9621 - val_loss: 0.1960 - val_accuracy: 0.9274 - val_precision: 0.9382 - val_recall: 0.9531 - val_auc: 0.9758
Epoch 3/10
134/134 [==============================] - 55s 409ms/step - loss: 0.1923 - accuracy: 0.9205 - precision: 0.9420 - recall: 0.9564 - auc: 0.9680 - val_loss: 0.1830 - val_accuracy: 0.9281 - val_precision: 0.9530 - val_recall: 0.9376 - val_auc: 0.9782
Epoch 4/10
134/134 [==============================] - 56s 415ms/step - loss: 0.1832 - accuracy: 0.9252 - precision: 0.9469 - recall: 0.9571 - auc: 0.9708 - val_loss: 0.1749 - val_accuracy: 0.9323 - val_precision: 0.9378 - val_recall: 